In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import sys
import torch
# conda activate Gseture
print('conda activate:', sys.prefix)
print("Python version:", sys.version)
print("cv2 version:", cv2.__version__)
print("mediapipe version", mp.__version__)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("torch version:", torch.__version__)

conda activate: c:\Users\gamja5th\anaconda3\envs\torch
Python version: 3.8.19 | packaged by conda-forge | (default, Mar 20 2024, 12:38:07) [MSC v.1929 64 bit (AMD64)]
cv2 version: 4.10.0
mediapipe version 0.10.10
pandas version: 2.0.3
numpy version: 1.24.1
torch version: 2.0.0+cu118


In [5]:
# from 경로 import class명
from main_files.CustomDataset import CustomDataset
from main_files.model import CNN_LSTM


# 필요한 모듈
import torch.nn as nn  
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm # 진행바
from torch.utils.data import Dataset,DataLoader ,random_split
from torchinfo import summary # 모델 요약
from tensorboardX import SummaryWriter # 텐서보드 (loss,accuracy 확인)

In [7]:
##
##   데이터셋 생성
##

dataset=CustomDataset(window_size=30, Folder_dir='./main_data/')

# train,test 분리
val_ratio=0.2
val_size=int(val_ratio*len(dataset))

train_size=len(dataset)-val_size
train_dataset, val_dataset=random_split(dataset,[train_size,val_size])


train_dataloader=DataLoader(train_dataset,batch_size=64,shuffle=True)  # shuffle: 미니배치들이 에폭마다 섞이는 유무.
val_dataloader=DataLoader(val_dataset,batch_size=64,shuffle=False)  # shuffle: 미니배치들이 에폭마다 섞이는 유무.

print("Dataset size:",len(dataset),'\n')
print("Traing data size:",len(train_dataset))
print("Validation data size:",len(val_dataset),'\n')   
print("Traing data # of batch:",len(train_dataloader))
print("Validation # of batch:",len(val_dataloader))

Dataset size: 3476 

Traing data size: 2781
Validation data size: 695 

Traing data # of batch: 44
Validation # of batch: 11


In [3]:
# gpu 설정


In [8]:
def train(model_name, train_loader,device,optimizer,loss_func,log_interval=10):
    model_name.train()
    Train_total_loss=0
    Train_correct_predictions=0

    for batch_idx,(x_train, y_train) in enumerate(tqdm(train_loader)):
        # cross entropy의 y는 LongTensor형이어야 함.
        y_train=y_train.type(torch.LongTensor)
        x_train=x_train.to(device)
        y_train=y_train.to(device)



        y_predict=model_name(x_train)
        
        # loss 계산
        loss=loss_func(y_predict,y_train.squeeze(dim=-1))
        Train_total_loss+=loss.item()

        # 정확도 계싼
        values, indices = torch.max(y_predict.data, dim=1,keepdim=True)
        Train_correct_predictions += (indices == y_train).sum().item()

        # 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch_idx % log_interval==0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch,batch_idx*len(x_train),len(train_dataloader.dataset),
        #         100.*batch_idx/len(train_dataloader.dataset),loss.item()
        #     ))

    avg_train_loss=Train_total_loss/len(train_loader.dataset)
    Train_accuracy = 100. * Train_correct_predictions / len(train_loader.dataset)
    print('Train Epoch: {} Average loss: {:.6f}, Accuracy: {:.2f}%'
          .format(epoch, avg_train_loss, Train_accuracy))



def evaluate(model_name,test_loader,device,loss_func):
    model_name.eval()
    correct=0
    val_loss=0
    with torch.no_grad():
        for idx,(x_test,y_test) in enumerate(test_loader):
            y_test=y_test.type(torch.LongTensor)
            x_test=x_test.to(device)
            y_test=y_test.to(device)  # torch.Size([64, 1])
            
            y_pred=model_name(x_test)
            
            val_loss+=loss_func(y_pred,y_test.squeeze(dim=-1)).item()

            # 정확하게 분류한 샘플 수 계산
            values, indices = torch.max(y_pred.data, dim=1,keepdim=True)        
            correct += (indices == y_test).sum().item()

            # print('y_test :',y_test.shape)     # torch.Size([64, 1])
            # print('indices: ',indices.shape)   # torch.Size([64, 1]) 

    avg_val_loss=val_loss/len(test_loader.dataset)
    accuracy = 100 * correct / len(test_loader.dataset)  # 정확도 계산
    print('Validation set: Average loss: {:.4f}, Accuracy: {:.2f}%'
          .format(avg_val_loss, accuracy))
    print()
        


In [10]:
device = torch.device('cuda:0')

# 모델 호출
CNN_LSTM_model=CNN_LSTM(
                input_size=99, 
                output_size=64,
                units=32).to(device)

# optimizer 설정
optimizer = optim.Adam(CNN_LSTM_model.parameters(), lr=0.0001)

# loss 함수
criterion = nn.CrossEntropyLoss()
# epoch 설정
epochs = 30

In [11]:
from torchinfo import summary

summary(CNN_LSTM_model, (64,30,99))

c:\Users\gamja5th\Documents\24_2_pioneer\code\Gesture_Recognition\main_files\model.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)


Layer (type:depth-idx)                   Output Shape              Param #
CNN_LSTM                                 [64, 4]                   --
├─Conv1d: 1-1                            [64, 64, 28]              19,072
├─ReLU: 1-2                              [64, 64, 28]              --
├─LSTM: 1-3                              [64, 28, 32]              12,544
├─Linear: 1-4                            [64, 4]                   132
├─Softmax: 1-5                           [64, 4]                   --
Total params: 31,748
Trainable params: 31,748
Non-trainable params: 0
Total mult-adds (M): 56.66
Input size (MB): 0.76
Forward/backward pass size (MB): 1.38
Params size (MB): 0.13
Estimated Total Size (MB): 2.27

In [12]:
gesture={
    0 : 'Palm',
    1 : 'Fist',
    2 : 'Finger Tip'
}

for epoch in range(epochs):
    
    train(
        model_name=CNN_LSTM_model, 
        train_loader=train_dataloader,
        optimizer=optimizer,
        loss_func=criterion,
        log_interval=1,
        device=device,)

    evaluate(
        model_name=CNN_LSTM_model,
        test_loader=val_dataloader,
        loss_func=criterion,
        device=device)

100%|██████████| 44/44 [00:00<00:00, 134.35it/s]


Train Epoch: 0 Average loss: 0.020200, Accuracy: 57.28%
Validation set: Average loss: 0.0193, Accuracy: 79.42%



100%|██████████| 44/44 [00:00<00:00, 414.24it/s]


Train Epoch: 1 Average loss: 0.018427, Accuracy: 89.61%
Validation set: Average loss: 0.0176, Accuracy: 97.99%



100%|██████████| 44/44 [00:00<00:00, 495.40it/s]


...

Train Epoch: 27 Average loss: 0.012084, Accuracy: 100.00%
Validation set: Average loss: 0.0121, Accuracy: 100.00%



100%|██████████| 44/44 [00:00<00:00, 487.59it/s]


Train Epoch: 28 Average loss: 0.012067, Accuracy: 100.00%
Validation set: Average loss: 0.0121, Accuracy: 100.00%



100%|██████████| 44/44 [00:00<00:00, 475.48it/s]

Train Epoch: 29 Average loss: 0.012052, Accuracy: 100.00%
Validation set: Average loss: 0.0121, Accuracy: 100.00%



In [13]:
torch.save(CNN_LSTM_model.state_dict(), './main_models/model_dict().pt')
torch.save(CNN_LSTM_model, './main_models/model.pt')

## 이해안되는 부분 확인중

In [ ]:
import torch

# 예측 출력을 나타내는 무작위 텐서 생성 (예시)
# 가정: 모델의 출력이 3개의 클래스를 분류하며, 배치 크기가 4인 경우
y_pred = torch.rand(4, 3)

# y_pred 텐서의 내용 확인
print(y_pred.shape)
print(y_pred)

# 각 샘플에 대한 최대 클래스 인덱스 찾기
values, indices = torch.max(y_pred,dim=1, keepdim=True)
# dim=1: 행을 따라 최대값 찾기, dim=0: 열을 따라 최대값 찾기
# keepdim=True: 출력 텐서각각을 크기가1인 차원으로 유지함.
# keepdim=False: 출력 텐서 각각의 크기가 1인 차원을 삭제함.


# predicted 텐서의 내용 확인
print(values)
print(indices)

torch.Size([4, 3])
tensor([[0.0168, 0.3880, 0.9685],
        [0.9958, 0.9044, 0.3451],
        [0.5561, 0.8403, 0.7750],
        [0.7458, 0.1477, 0.5786]])
tensor([[0.9685],
        [0.9958],
        [0.8403],
        [0.7458]])
tensor([[2],
        [0],
        [1],
        [0]])


## 배움

1. .item()  : 텐서의 값을 일반 파이썬 스칼라값(float등)으로 변환해줌

2. crossentropy수행시 y의 값은 LongTensor (=int=정수형) 로 들어가야함.

3. torch.max : 분류문제에서 정확도및 loss값 확인하려고 사용함

4. - len(test_loader.dataset) : 전체 데이터 셋의 개수, 
   - len(test_loader) : 하나의 배치의 수.

5. torchinfo : 모델정보를 볼수 있음. 
   - pip install torchinfo
   - from torchinfo import summary
   - summary(model_name , (batch size, input size))